<a href="https://colab.research.google.com/github/shunrei9841-sudo/Guadalupe/blob/main/Parcial%202-7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from scipy.stats import expon

# --- Parámetros del Problema y Simulación ---
lambda_Y = 1.0  # Parámetro de Y
Threshold = 2.0 # Umbral de la suma (c = 2.0)
N = 1000000     # Número de simulaciones

# --- Generación de muestras ---
U = np.random.uniform(0, 1, N)

# Generar X: X = 2 * sqrt(U)
X_samples = 2 * np.sqrt(U)

# Generar Y: Y = -1/lambda_Y * log(1 - U)
Y_samples = -1/lambda_Y * np.log(1 - U)

# --- A) Simulación Simple (Monte Carlo Directo) ---
print("--- A) Simulación Simple ---")

Sum_simple = X_samples + Y_samples
indicator_simple = (Sum_simple > Threshold).astype(int)

# Estimador P_hat = E[I(X + Y > c)]
P_hat_simple = np.mean(indicator_simple)

# Varianza del estimador simple
Var_indicator_simple = np.var(indicator_simple, ddof=1)
Var_P_hat_simple = Var_indicator_simple / N

print(f"Estimación P(X + Y > 2) (Simple): {P_hat_simple:.6f}")
print(f"Varianza del Estimador (Simple): {Var_P_hat_simple:.10f}")
print("-" * 50)


# --- B) Condicionamiento sobre X: E[I(X + Y > c) | X] ---
print("--- B) Condicionamiento sobre X ---")

# Estimador de condicionamiento: E[I(X + Y > c)] = E[E[I(X + Y > c) | X]]
# E[I(X + Y > c) | X=x] = P(Y > c - x | X=x)
# Como X e Y son independientes: P(Y > c - x)

# P(Y > c - x) es la Función de Supervivencia (SF) de Y evaluada en (c - x):
# S_Y(c - x) = exp(-lambda_Y * (c - x)), para c - x > 0 (es decir, x < c)
# Si x >= c, P(Y > c - x) = P(Y > negativo) = 1.

# 1. Usamos las mismas muestras X_samples
X_cond = X_samples
c = Threshold

# 2. Calcular el estimador condicional g_cond(X) = E[I(X + Y > c) | X]
g_cond_X = np.zeros(N)

# Caso 1: x < c (x < 2). P(Y > c - x) = exp(-(c - x))
idx_lt_c = X_cond < c
g_cond_X[idx_lt_c] = np.exp(-lambda_Y * (c - X_cond[idx_lt_c]))

# Caso 2: x >= c (x >= 2). P(Y > c - x) = 1 (ya que c - x <= 0)
idx_ge_c = X_cond >= c
g_cond_X[idx_ge_c] = 1.0

# 3. Estimar la probabilidad P(X + Y > 2) con el estimador condicional
P_hat_cond = np.mean(g_cond_X)

# 4. Calcular la varianza del estimador condicional
Var_g_cond_X = np.var(g_cond_X, ddof=1)
Var_P_hat_cond = Var_g_cond_X / N

# 5. Porcentaje de Reducción de Varianza
Reduccion_cond = (Var_P_hat_simple - Var_P_hat_cond) / Var_P_hat_simple * 100

print(f"Estimación P(X + Y > 2) (Condicionamiento): {P_hat_cond:.6f}")
print(f"Varianza del Estimador (Condicionamiento): {Var_P_hat_cond:.10f}")
print(f"Reducción de Varianza: {Reduccion_cond:.2f}%")
print("-" * 50)


# --- C) Variable de Control: C = X + Y ---
print("--- C) Variable de Control C = X + Y ---")

# El estimador de la variable de control es: g_ctrl = g(X, Y) - b * (C - E[C])
# Donde g(X, Y) = I(X + Y > c) y C = X + Y.
# El valor esperado E[C] es conocido: E[C] = E[X] + E[Y].

# 1. Calcular E[X] y E[Y] analíticamente
# E[Y] = 1/lambda_Y = 1/1 = 1.0
E_Y_analytic = 1.0

# E[X] = Integral de x * f(x) dx de 0 a 2
# E[X] = Integral de x * (x/2) dx = Integral de (x^2/2) dx de 0 a 2
# E[X] = [x^3/6]_0^2 = 8/6 = 4/3 ≈ 1.333333
E_X_analytic = 4/3

# E[C] = E[X] + E[Y]
E_C_analytic = E_X_analytic + E_Y_analytic

# 2. Calcular el factor b* óptimo (usando valores muestrales)
# b* = Cov[g(X, Y), C] / Var[C]
g_XY = indicator_simple # Estimador de interés
C_samples = Sum_simple   # Variable de control

# Calcular Cov[g(XY), C]
Cov_matrix = np.cov(g_XY, C_samples)
Cov_gXY_C = Cov_matrix[0, 1]

# Calcular Var[C]
Var_C_sample = np.var(C_samples, ddof=1)

# Calcular el factor b* óptimo
b_optimo = Cov_gXY_C / Var_C_sample

# 3. Calcular el estimador g_ctrl
g_ctrl = g_XY - b_optimo * (C_samples - E_C_analytic)

# 4. Estimar la probabilidad P(X + Y > 2) con el estimador de control
P_hat_ctrl = np.mean(g_ctrl)

# 5. Calcular la varianza del estimador de control
Var_g_ctrl = np.var(g_ctrl, ddof=1)
Var_P_hat_ctrl = Var_g_ctrl / N

# 6. Porcentaje de Reducción de Varianza
Reduccion_ctrl = (Var_P_hat_simple - Var_P_hat_ctrl) / Var_P_hat_simple * 100

print(f"E[X] = {E_X_analytic:.4f}, E[Y] = {E_Y_analytic:.4f}, E[C] = {E_C_analytic:.4f}")
print(f"Valor óptimo b* (Muestra): {b_optimo:.6f}")
print(f"Estimación P(X + Y > 2) (Control): {P_hat_ctrl:.6f}")
print(f"Varianza del Estimador (Control): {Var_P_hat_ctrl:.10f}")
print(f"Reducción de Varianza: {Reduccion_ctrl:.2f}%")

--- A) Simulación Simple ---
Estimación P(X + Y > 2) (Simple): 0.533409
Varianza del Estimador (Simple): 0.0000002489
--------------------------------------------------
--- B) Condicionamiento sobre X ---
Estimación P(X + Y > 2) (Condicionamiento): 0.568314
Varianza del Estimador (Condicionamiento): 0.0000000550
Reducción de Varianza: 77.89%
--------------------------------------------------
--- C) Variable de Control C = X + Y ---
E[X] = 1.3333, E[Y] = 1.0000, E[C] = 2.3333
Valor óptimo b* (Muestra): 0.270461
Estimación P(X + Y > 2) (Control): 0.532516
Varianza del Estimador (Control): 0.0000001048
Reducción de Varianza: 57.88%
